In [1]:
from cadcad.spaces import space
from cadcad.dynamics import block
from cadcad.spaces import Bit, Real, Integer
from cadcad.points import Point

import numpy as np

In [2]:
R2 = Real**2

R2dims = list(R2.dimensions().keys())
R2dims

['real_0', 'real_1']

In [3]:
CartesianPlane = R2.rename_dims({R2dims[0]:'x', R2dims[1]:'y'})

CartesianPlane.dimensions()

{'x': 'Real', 'y': 'Real'}

In [4]:
PolarPlane = R2.rename_dims({R2dims[0]:'r', R2dims[1]:'theta'})

PolarPlane.dimensions()

{'r': 'Real', 'theta': 'Real'}

In [5]:
PolarPlane.unroll_schema()

{'r': {'real': 'float'}, 'theta': {'real': 'float'}}

In [6]:
CartesianPlane.unroll_schema()

{'x': {'real': 'float'}, 'y': {'real': 'float'}}

In [7]:
x0y1 = Point(CartesianPlane, {'x':{'real':0.0}, 'y':{'real':1.0}})

In [8]:
print(x0y1)

Point in space 2-Real has data
{
    "x": {
        "real": 0.0
    },
    "y": {
        "real": 1.0
    }
}



In [9]:
@block
def cartesian2polar(domain: Point[CartesianPlane]) -> Point[PolarPlane]:
    
    x = domain["x"]["real"]
    y = domain["y"]["real"]

    r = np.sqrt(x**2+y**2)
    if x==0:
        theta = np.sign(y)*np.pi/2
    else:
        theta = np.arctan(y/x)

    output = {"r":{"real":r}, "theta":{"real":theta} }

    return Point(PolarPlane, output)

@block
def polar2cartesian(domain: Point[PolarPlane]) -> Point[CartesianPlane]:
    
    r = domain["r"]["real"]
    theta = domain["theta"]["real"]

    x = r*np.cos(theta)
    y = r*np.sin(theta)

    output = {"x":{"real":x}, "y":{"real":y} }

    return Point(CartesianPlane, output)

In [10]:
cartesian2polar

Block(_Block__function=<function cartesian2polar at 0x7fbd50a51a20>, _Block__domain=cadcad.points.Point[cadcad.spaces.space.<locals>.NewSpace], _Block__codomain=cadcad.points.Point[cadcad.spaces.space.<locals>.NewSpace], _Block__param_space=None)

In [11]:
cartesian2polar(x0y1)

In [12]:
print(cartesian2polar(x0y1))

Point in space 2-Real has data
{
    "r": {
        "real": 1.0
    },
    "theta": {
        "real": 1.5707963267948966
    }
}



In [13]:
print(polar2cartesian(cartesian2polar(x0y1)))

Point in space 2-Real has data
{
    "x": {
        "real": 6.123233995736766e-17
    },
    "y": {
        "real": 1.0
    }
}



note that `1.5707963267948966` radians is `90` degrees. This leads me to want to rename the dimensions of this space such that

```
{
    "r": {
        "real": 1.0
    },
    "theta": {
        "radians": 1.5707963267948966
    }
}
```
because then i can make a slightly different space which has
```
{
    "r": {
        "real": 1.0
    },
    "theta": {
        "degrees": 90.0
    }
}
```
this helps us understand why there are benefits to using `Real`, instead of just `float`, the keys give us a way of clarifying units, and even encoding blocks to do unit conversions

In [14]:
Radians = Real.rename_dims({'real':'radians'})
Degrees = Real.rename_dims({'real':'degrees'})

In [15]:
Radians.unroll_schema()

{'radians': 'float'}

In [16]:
@block
def radians2degrees(domain: Point[Radians]) -> Point[Degrees]:
    
    input = domain["radians"]
    
    output = input*180.0/np.pi

    return Point(Degrees, {'degrees':output})

@block
def degrees2radians(domain: Point[Degrees]) -> Point[Radians]:
    
    input = domain["degrees"]
    
    output = input*np.pi/180.0

    return Point(Radians, {'radians':output})

In [17]:
deg90 = Point(Degrees, {'degrees': 90.0})

In [18]:
deg90

In [19]:
type(deg90)

cadcad.points.Point

In [20]:
deg90.data

{'degrees': 90.0}

In [21]:
print(deg90)

Point in space Real has data
{
    "degrees": 90.0
}



In [22]:
radpt = degrees2radians(deg90)

In [23]:
print(radpt)

Point in space Real has data
{
    "radians": 1.5707963267948966
}



In [24]:
degpt = radians2degrees(radpt)

In [25]:
print(degpt)

Point in space Real has data
{
    "degrees": 90.0
}

